In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt 
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer #Data transformation
from sklearn.model_selection import train_test_split #Data testing
from sklearn.linear_model import LogisticRegression #Prediction Model
from sklearn.metrics import accuracy_score #Comparison between real and predicted
from wordcloud import WordCloud #Word visualization
from nltk import word_tokenize
nltk.download('stopwords')
import numpy as np 
import pandas as pd

# Data Gatehering

In [ ]:
data=pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv',sep = ',', names=['id','country','Label','Text'])
validation=pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv',sep = ',', names=['id','Company','Label','Text'])

In [ ]:
#data.head()
print(validation.head(),"\n\n\n\n\n\n\n",data.head())

In [ ]:

data=data.drop("id", axis='columns')

validation=validation.drop("id", axis='columns')

In [ ]:
print(data.info(),"\n\n\n\n\n\n\n\n\n\n\n\n\n",validation.info())

In [ ]:
print(data.isnull().sum(),"\n\n\n\n\n\n\n\n",validation.isnull().sum())

In [ ]:
print(data['country'].unique(),"\n\n\n\n\n",validation['Company'].unique())

In [ ]:
# Check for null values in the 'text' column
null_values = data['Text'].isnull()

# Display the rows with null values in the 'text' column
rows_with_null = data[null_values]
print(rows_with_null)

In [ ]:
# Replace missing values with mode
column_name = 'Text'
mode_value = data[column_name].mode()[0]
data[column_name].fillna(mode_value, inplace=True)
data.isnull().sum()

# Data Preprocessing

In [ ]:
## Text transformation
data["lower"]=data.Text.str.lower() #lowercase
data["lower"]=[str(data) for data in data.lower] #converting all to string
data["lower"]=data.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex
validation["lower"]=validation.Text.str.lower() #lowercase
validation["lower"]=[str(data) for data in validation.lower] #converting all to stringv
validation["lower"]=validation.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex

In [ ]:
validation

# Positive Wordcloud on Label

In [ ]:
word_cloud_text = ''.join(data[data["Label"]=="Positive"].lower)
#Creation of wordcloud
wordcloud = WordCloud(
    max_font_size=100,
    max_words=100,
    background_color="black",
    scale=10,
    width=800,
    height=800
).generate(word_cloud_text)
#Figure properties
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Negative Wordcloud

In [ ]:
word_cloud_text = ''.join(data[data["Label"]=="Negative"].lower)
#Creation of wordcloud
wordcloud = WordCloud(
    max_font_size=100,
    max_words=100,
    background_color="black",
    scale=10,
    width=800,
    height=800
).generate(word_cloud_text)
#Figure properties
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Irrelevant Wordcloud on the basis of label columns

In [ ]:
word_cloud_text = ''.join(data[data["Label"]=="Irrelevant"].lower)
#Creation of wordcloud
wordcloud = WordCloud(
    max_font_size=100,
    max_words=100,
    background_color="black",
    scale=10,
    width=800,
    height=800
).generate(word_cloud_text)
#Figure properties
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
#Count information per category
plot1=data.groupby(by=["country","Label"]).count().reset_index()
plot1

# Count number on positive or negative or neutral or irrevelent

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.countplot(x='Label', data = data)

# Label on Data

In [ ]:
fig = plt.figure(figsize=(7,7))
colors = ("yellowgreen", "gold", "red","blue")
wp = {'linewidth':2, 'edgecolor':"black"}
tags = data['Label'].value_counts()
explode = (0.1,0.1,0.1,0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors = colors,
         startangle=90, wedgeprops = wp, explode = explode, label='')
plt.title('Distribution of sentiments')

# Validation

In [ ]:
fig = plt.figure(figsize=(7,7))
colors = ("yellowgreen", "gold", "red","blue")
wp = {'linewidth':2, 'edgecolor':"black"}
tags = validation['Label'].value_counts()
explode = (0.1,0.1,0.1,0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors = colors,
         startangle=90, wedgeprops = wp, explode = explode, label='')
plt.title('Distribution of sentiments')

In [ ]:
validation

In [ ]:
#Figure of comparison per branch
plt.figure(figsize=(20,6))
sns.barplot(data=plot1,x="country",y="Text",hue="Label")
plt.xticks(rotation=90)
plt.xlabel("Brand")
plt.ylabel("Number of tweets")
plt.grid()
plt.title("Distribution of tweets per Branch and Type");

In [ ]:
texts = data['Text']

In [ ]:
text_lens = [len(t.split()) for t in texts.values]
len_mean = np.mean(text_lens)
len_mean

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 8))
axes[0].set_title('Distribution of number of tokens in tweets')
sns.boxplot(text_lens, ax=axes[0])
sns.histplot(text_lens, bins=100, kde=True, ax=axes[1])
axes[1].vlines(len_mean, 0, 5000, color = 'r')
plt.annotate("mean", xy=(len_mean, 5000), xytext=(len_mean-2, 5050),
            color='r')
plt.show()

# fIND Unique Words

In [ ]:
#Choosing english stopwords
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')
stop_words[:5]

In [ ]:
#Initial Bag of Words
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    stop_words=stop_words, #English Stopwords
    ngram_range=(1, 1) #analysis of one word
)
bow_counts

In [ ]:
#Train - Test splitting
reviews_train, reviews_test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
#Labels for train and test encoding
y_train_bow = reviews_train['Label']
y_test_bow = reviews_test['Label']

In [ ]:
#Creation of encoding related to train dataset
X_train_bow = bow_counts.fit_transform(reviews_train.lower)
#Transformation of test dataset with train encoding
X_test_bow = bow_counts.transform(reviews_test.lower)

In [ ]:
# Logistic regression
model1 = LogisticRegression(C=1, solver="liblinear",max_iter=200)
model1.fit(X_train_bow, y_train_bow)
# Prediction
test_pred = model1.predict(X_test_bow)
print("Accuracy: ", accuracy_score(y_test_bow, test_pred) * 100)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
print(confusion_matrix(y_test_bow, test_pred))
print("\n")
print(classification_report(y_test_bow, test_pred))